# Our Dataset Image Classificaiton

In [3]:
### Reference from: https://github.com/yysijie/st-gcn/blob/master/net/utils/graph.py

import os
import torch
import numpy as np


class Graph:
    """The Graph to model the skeletons extracted by the Alpha-Pose.
    Args:
        - strategy: (string) must be one of the follow candidates
            - uniform: Uniform Labeling,
            - distance: Distance Partitioning,
            - spatial: Spatial Configuration,
        For more information, please refer to the section 'Partition Strategies'
            in our paper (https://arxiv.org/abs/1801.07455).
        - layout: (string) must be one of the follow candidates
            - coco_cut: Is COCO format but cut 4 joints (L-R ears, L-R eyes) out.
        - max_hop: (int) the maximal distance between two connected nodes.
        - dilation: (int) controls the spacing between the kernel points.
    """
    def __init__(self,
                 layout='coco_cut',
                 strategy='uniform',
                 max_hop=1,
                 dilation=1):
        self.max_hop = max_hop
        self.dilation = dilation

        self.get_edge(layout)
        self.hop_dis = get_hop_distance(self.num_node, self.edge, max_hop)
        self.get_adjacency(strategy)

    def get_edge(self, layout):
        if layout == 'coco_cut':
            self.num_node = 14
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(6, 4), (4, 2), (2, 13), (13, 1), (5, 3), (3, 1), (12, 10),
                             (10, 8), (8, 2), (11, 9), (9, 7), (7, 1), (13, 0)]
            self.edge = self_link + neighbor_link
            self.center = 13
            
        elif layout == 'mediapipe_KSL':
            self.num_node = 47
            self_link = [(i, i) for i in range(self.num_node)]
            # used_key_points=
            # [0,11,12,13,14]+[i for i in range(33,33+21)] + [i for i in range(54,54+21)] 
            neighbor_link = [(0,1),(0,2),(1,3),(2,4),(3,26),(4,5), # nose-arms-wrist
                             
                             (5,6),(6,7),(7,8),(8,9),
                             (5,10),(10,11),(11,12),(12,13),
                             (5,14),(14,15),(15,16),(16,17),
                             (5,18),(18,19),(19,20),(20,21),
                             (5,22),(22,23),(23,24),(24,25),
                             
                             (26,27),(27,28),(28,29),(29,30),
                             (26,31),(31,32),(32,33),(33,34),
                             (26,35),(35,36),(36,37),(37,38),
                             (26,39),(39,40),(40,41),(41,42),
                             (26,43),(43,44),(44,45),(45,46),
                            ]
            self.edge = self_link + neighbor_link
            self.center = 0
        else:
            raise ValueError('This layout is not supported!')

    def get_adjacency(self, strategy):
        valid_hop = range(0, self.max_hop + 1, self.dilation)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis ==
                                                                hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
            #self.A = np.swapaxes(np.swapaxes(A, 0, 1), 1, 2)
        else:
            raise ValueError("This strategy is not supported!")


def get_hop_distance(num_node, edge, max_hop=1):
    A = np.zeros((num_node, num_node))
    for i, j in edge:
        A[j, i] = 1
        A[i, j] = 1

    # compute hop steps
    hop_dis = np.zeros((num_node, num_node)) + np.inf
    transfer_mat = [np.linalg.matrix_power(A, d) for d in range(max_hop + 1)]
    arrive_mat = (np.stack(transfer_mat) > 0)
    for d in range(max_hop, -1, -1):
        hop_dis[arrive_mat[d]] = d
    return hop_dis


def normalize_digraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    AD = np.dot(A, Dn)
    return AD


def normalize_undigraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-0.5)
    DAD = np.dot(np.dot(Dn, A), Dn)
    return DAD

In [4]:
### Reference from: https://github.com/yysijie/st-gcn/tree/master/net

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# from Actionsrecognition.Utils import Graph


class GraphConvolution(nn.Module):
    """The basic module for applying a graph convolution.
    Args:
        - in_channel: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (int) Size of the graph convolving kernel.
        - t_kernel_size: (int) Size of the temporal convolving kernel.
        - t_stride: (int, optional) Stride of the temporal convolution. Default: 1
        - t_padding: (int, optional) Temporal zero-padding added to both sides of
            the input. Default: 0
        - t_dilation: (int, optional) Spacing between temporal kernel elements. Default: 1
        - bias: (bool, optional) If `True`, adds a learnable bias to the output.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math:`(N, in_channels, T_{in}, V)`,
                 A: Graph adjacency matrix in :math:`(K, V, V)`,
        - Output: Graph sequence out in :math:`(N, out_channels, T_{out}, V)`

            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.

    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 t_kernel_size=1,
                 t_stride=1,
                 t_padding=0,
                 t_dilation=1,
                 bias=True):
        super().__init__()

        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels,
                              out_channels * kernel_size,
                              kernel_size=(t_kernel_size, 1),
                              padding=(t_padding, 0),
                              stride=(t_stride, 1),
                              dilation=(t_dilation, 1),
                              bias=bias)

    def forward(self, x, A):
        x = self.conv(x)
        n, kc, t, v = x.size()
        x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
        x = torch.einsum('nkctv,kvw->nctw', (x, A))

        return x.contiguous()


class st_gcn(nn.Module):
    """Applies a spatial temporal graph convolution over an input graph sequence.
    Args:
        - in_channels: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (tuple) Size of the temporal convolving kernel and
            graph convolving kernel.
        - stride: (int, optional) Stride of the temporal convolution. Default: 1
        - dropout: (int, optional) Dropout rate of the final output. Default: 0
        - residual: (bool, optional) If `True`, applies a residual mechanism.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math: `(N, in_channels, T_{in}, V)`,
                 A: Graph Adjecency matrix in :math: `(K, V, V)`,
        - Output: Graph sequence out in :math: `(N, out_channels, T_{out}, V)`
            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.
    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1,
                 dropout=0,
                 residual=True):
        super().__init__()
        assert len(kernel_size) == 2
        assert kernel_size[0] % 2 == 1
        #print(kernel_size)(9, 3)
        padding = ((kernel_size[0] - 1) // 2, 0)
        #print(padding)(4, 0)

        self.gcn = GraphConvolution(in_channels, out_channels, kernel_size[1])
        self.tcn = nn.Sequential(nn.BatchNorm2d(out_channels),
                                 nn.ReLU(inplace=True),
                                 nn.Conv2d(out_channels,
                                           out_channels,
                                           (kernel_size[0], 1),
                                           (stride, 1),
                                           padding),
                                 nn.BatchNorm2d(out_channels),
                                 nn.Dropout(dropout, inplace=True),
                                 )

        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and (stride == 1):
            self.residual = lambda x: x
        else:
            self.residual = nn.Sequential(nn.Conv2d(in_channels,
                                                    out_channels,
                                                    kernel_size=1,
                                                    stride=(stride, 1)),
                                          nn.BatchNorm2d(out_channels)
                                          )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, A):
        res = self.residual(x)
        # print(res)
        x = self.gcn(x, A)
        #print("x_in:",x.size())
        x = self.tcn(x) + res
        #print("x_out:",x.size())
        return self.relu(x)


class StreamSpatialTemporalGraph(nn.Module):
    """Spatial temporal graph convolutional networks.
    Args:
        - in_channels: (int) Number of input channels.
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs. If `None` return pooling features of
            the last st-gcn layer instead.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :math:`(N, in_channels, T_{in}, V_{in})`
        - Output: :math:`(N, num_class)` where
            :math:`N` is a batch size,
            :math:`T_{in}` is a length of input sequence,
            :math:`V_{in}` is the number of graph nodes,
        or If num_class is `None`: `(N, out_channels)`
            :math:`out_channels` is number of out_channels of the last layer.
    """
    def __init__(self, in_channels, graph_args, num_class=None,
                 edge_importance_weighting=True, **kwargs):
        super().__init__()
        # Load graph.
        graph = Graph(**graph_args)
        A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
        self.register_buffer('A', A)

        # Networks.
        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 9
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        kwargs0 = {k: v for k, v in kwargs.items() if k != 'dropout'}

        self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
        self.st_gcn_networks = nn.ModuleList((
            st_gcn(in_channels, 64, kernel_size, 1, residual=False, **kwargs0),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 128, kernel_size, 2, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 256, kernel_size, 2, **kwargs),
            st_gcn(256, 256, kernel_size, 1, **kwargs),
            st_gcn(256, 256, kernel_size, 1, **kwargs)
        ))

        # initialize parameters for edge importance weighting.
        if edge_importance_weighting:
            self.edge_importance = nn.ParameterList([
                nn.Parameter(torch.ones(A.size()))
                for i in self.st_gcn_networks
            ])
        else:
            self.edge_importance = [1] * len(self.st_gcn_networks)

        if num_class is not None:
            self.cls = nn.Conv2d(256, num_class, kernel_size=1)
        else:
            self.cls = lambda x: x

    def forward(self, x):
        # data normalization.
        N, C, T, V = x.size()
        x = x.permute(0, 3, 1, 2).contiguous()  # (N, V, C, T)
        x = x.view(N, V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(N, C, T, V)

        # forward.
        for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
            x = gcn(x, self.A * importance)

        x = F.avg_pool2d(x, x.size()[2:])
        x = self.cls(x)
        x = x.view(x.size(0), -1)

        return x


class TwoStreamSpatialTemporalGraph(nn.Module):
    """Two inputs spatial temporal graph convolutional networks.
    Args:
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :tuple of math:`((N, 3, T, V), (N, 2, T, V))`
        for points and motions stream where.
            :math:`N` is a batch size,
            :math:`in_channels` is data channels (3 is (x, y, score)), (2 is (mot_x, mot_y))
            :math:`T` is a length of input sequence,
            :math:`V` is the number of graph nodes,
        - Output: :math:`(N, num_class)`
    """
    def __init__(self, graph_args, num_class, edge_importance_weighting=True,
                 **kwargs):
        super().__init__()
        self.pts_stream = StreamSpatialTemporalGraph(3, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)
        self.mot_stream = StreamSpatialTemporalGraph(3, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)

        # self.fcn = nn.Linear(256 * 2, num_class)
        self.fcn = nn.Linear(256 , num_class)
        
        # self.atten1 = nn.Linear(256 * 2, 128)
        # self.atten_bn = nn.BatchNorm1d(128)
        # self.atten_relu= nn.ReLU(inplace=True)
        # self.atten2 = nn.Linear(128,32)
        # self.atten_relu2= nn.ReLU(inplace=True)
        # self.atten3 = nn.Linear(32, 256 * 2)
        # self.atten_act = nn.Sigmoid() 
        
    def forward(self, inputs):
        out1 = self.pts_stream(inputs[0])
        # out2 = self.mot_stream(inputs[1])
        
        #print(out1.size())torch.Size([32, 256])
        #print(out2.size())torch.Size([32, 256])
        # concat = torch.cat([out1, out2], dim=-1)
        
        # attn = self.atten1(concat)
        # attn = self.atten_bn(attn)
        # attn = self.atten_relu(attn)
        # attn = self.atten2(attn)
        # attn = self.atten_relu2(attn)
        # attn = self.atten3(attn)
        # attn = self.atten_act(attn)
        # concat = concat * attn
        
        
        # out = self.fcn(concat)
        out = self.fcn(out1)
        
        return out
        # return torch.sigmoid(out)
        #return F.softmax(out,dim=-1)

In [8]:
num_class=20

graph_args = {'layout':'mediapipe_KSL','strategy': 'spatial'}
model = TwoStreamSpatialTemporalGraph(graph_args, num_class)
a=torch.randn(1,3,40,47)
b=torch.randn(1,3,40,47)
# print(model)
out = model((a,b))
print(out.shape)


torch.Size([1, 20])


In [27]:
import os
import time
import torch
import pickle
import numpy as np
import torch.nn.functional as F
from shutil import copyfile
from tqdm import tqdm
from torch.utils import data
from torch.optim.adadelta import Adadelta
from sklearn.model_selection import train_test_split

#device = 'cuda'
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("using", device, "device")

epochs = 30
batch_size = 32 #32

def load_dataset(data_files, batch_size, split_size=0.2,used_key_points=None):#0.2
    """Load data files into torch DataLoader with/without spliting train-test.
    """
    features, labels = [], []
    for fil in data_files:
        with open(fil, 'rb') as f:
            fts, lbs = pickle.load(f)
            features.append(fts)
            labels.append(lbs)
        del fts, lbs
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    if used_key_points != None:
        features = features[:,:,:,used_key_points]
    print(features.shape)
    print(labels.shape)

    if split_size > 0:
        x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=split_size,random_state=0,stratify=labels)
        
        train_set = data.TensorDataset(torch.tensor(x_train, dtype=torch.float32),torch.tensor(y_train, dtype=torch.int64))
        valid_set = data.TensorDataset(torch.tensor(x_valid, dtype=torch.float32),torch.tensor(y_valid, dtype=torch.int64))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = data.DataLoader(valid_set, batch_size)
    else:
        train_set = data.TensorDataset(torch.tensor(features, dtype=torch.float32),torch.tensor(labels, dtype=torch.int64))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = None
    return train_loader, valid_loader

def accuracy_batch(y_pred, y_true):
    # print(y_pred.shape,y_true.shape)
    # return (y_pred.argmax(1) == y_true.argmax(1)).mean()
    return (y_pred.argmax(1) == y_true).mean()


def set_training(model, mode=True):
    for p in model.parameters():
        p.requires_grad = mode
    model.train(mode)
    return model

save_folder = os.path.join(os.environ['HOME'],"KSL_V2/Outputs")
os.makedirs(save_folder,exist_ok=True)
used_key_points=[0,11,12,13,14]+[i for i in range(33,33+21)] + [i for i in range(54,54+21)] 
train_loader, valid_loader = load_dataset([os.path.join(os.environ['HOME'],"KSL_V2/Datasets/KSL_1_dataset.pkl")], 32,0.2,used_key_points) #batch_size = 32
dataloader = {'train': train_loader, 'valid': valid_loader}
num_class=20

graph_args = {'layout':'mediapipe_KSL','strategy': 'spatial'}
model = TwoStreamSpatialTemporalGraph(graph_args, num_class).to(device)
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"number of params: {n_parameters}")
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
losser = torch.nn.CrossEntropyLoss()

loss_list = {'train': [], 'valid': []}
accu_list = {'train': [], 'valid': []}
best_acc = -1
for e in range(epochs):
    print('Epoch {}/{}'.format(e, epochs - 1))
    for phase in ['train', 'valid']:
        if phase == 'train':
            model = set_training(model, True)
        else:
            model = set_training(model, False)

        run_loss = 0.0
        run_accu = 0.0
        with tqdm(dataloader[phase], desc=phase) as iterator:
            for pts, lbs in iterator:
                # Create motion input by distance of points (x, y) of the same node
                # in two frames.
                mot = pts[:, :, 1:, :] - pts[:, :, :-1, :]

                mot = mot.to(device)
                pts = pts.to(device)
                lbs = lbs.to(device)
                
                # Forward.
                out = model((pts, mot))
                #print(lbs)

                #print(out)
                loss = losser(out, lbs)

                if phase == 'train':
                    # Backward.
                    model.zero_grad()
                    loss.backward()
                    optimizer.step()

                run_loss += loss.item()
                accu = accuracy_batch(out.detach().cpu().numpy(),
                                      lbs.detach().cpu().numpy())
                run_accu += accu

                iterator.set_postfix_str(' loss: {:.4f}, accu: {:.4f}'.format(
                    loss.item(), accu))
                iterator.update()
                #break
        loss_list[phase].append(run_loss / len(iterator))
        accu_list[phase].append(run_accu / len(iterator))
        #print(accu_list)
        #print(torch.max(accu_list))
    if(best_acc < accu_list['valid'][-1]):
        best_acc = accu_list['valid'][-1]
        torch.save(model.state_dict(), os.path.join(save_folder, 'tsstg-model_best.pth'))
        #break

    print('Summary epoch:\n - Train loss: {:.4f}, accu: {:.4f}\n - Valid loss:'
          ' {:.4f}, accu: {:.4f}'.format(loss_list['train'][-1], accu_list['train'][-1],
                                         loss_list['valid'][-1], accu_list['valid'][-1]))
del model
# 0.9655696902654868

using cuda device
(129459, 3, 1, 47)
(129459,)
number of params: 6267268
Epoch 0/29


valid: 100%|██████████| 810/810 [00:03<00:00, 220.73it/s,  loss: 1.0722, accu: 0.7500]


Summary epoch:
 - Train loss: 0.6597, accu: 0.7754
 - Valid loss: 0.3473, accu: 0.8924
Epoch 1/29


valid: 100%|██████████| 810/810 [00:03<00:00, 220.69it/s,  loss: 0.7393, accu: 0.7500]


Summary epoch:
 - Train loss: 0.1868, accu: 0.9371
 - Valid loss: 0.2266, accu: 0.9503
Epoch 2/29


...

Summary epoch:
 - Train loss: 0.0185, accu: 0.9939
 - Valid loss: 0.0279, accu: 0.9925
Epoch 28/29


valid: 100%|██████████| 810/810 [00:03<00:00, 221.13it/s,  loss: 0.0199, accu: 1.0000]


Summary epoch:
 - Train loss: 0.0188, accu: 0.9940
 - Valid loss: 0.0283, accu: 0.9931
Epoch 29/29


valid: 100%|██████████| 810/810 [00:03<00:00, 220.09it/s,  loss: 0.0095, accu: 1.0000]

Summary epoch:
 - Train loss: 0.0180, accu: 0.9942
 - Valid loss: 0.0248, accu: 0.9927


# Our Dataset Video Classification

In [28]:
### Reference from: https://github.com/yysijie/st-gcn/blob/master/net/utils/graph.py

import os
import torch
import numpy as np


class Graph:
    """The Graph to model the skeletons extracted by the Alpha-Pose.
    Args:
        - strategy: (string) must be one of the follow candidates
            - uniform: Uniform Labeling,
            - distance: Distance Partitioning,
            - spatial: Spatial Configuration,
        For more information, please refer to the section 'Partition Strategies'
            in our paper (https://arxiv.org/abs/1801.07455).
        - layout: (string) must be one of the follow candidates
            - coco_cut: Is COCO format but cut 4 joints (L-R ears, L-R eyes) out.
        - max_hop: (int) the maximal distance between two connected nodes.
        - dilation: (int) controls the spacing between the kernel points.
    """
    def __init__(self,
                 layout='coco_cut',
                 strategy='uniform',
                 max_hop=1,
                 dilation=1):
        self.max_hop = max_hop
        self.dilation = dilation

        self.get_edge(layout)
        self.hop_dis = get_hop_distance(self.num_node, self.edge, max_hop)
        self.get_adjacency(strategy)

    def get_edge(self, layout):
        if layout == 'coco_cut':
            self.num_node = 14
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(6, 4), (4, 2), (2, 13), (13, 1), (5, 3), (3, 1), (12, 10),
                             (10, 8), (8, 2), (11, 9), (9, 7), (7, 1), (13, 0)]
            self.edge = self_link + neighbor_link
            self.center = 13
            
        elif layout == 'mediapipe_KSL':
            self.num_node = 47
            self_link = [(i, i) for i in range(self.num_node)]
            # used_key_points=
            # [0,11,12,13,14]+[i for i in range(33,33+21)] + [i for i in range(54,54+21)] 
            neighbor_link = [(0,1),(0,2),(1,3),(2,4),(3,26),(4,5), # nose-arms-wrist
                             
                             (5,6),(6,7),(7,8),(8,9),
                             (5,10),(10,11),(11,12),(12,13),
                             (5,14),(14,15),(15,16),(16,17),
                             (5,18),(18,19),(19,20),(20,21),
                             (5,22),(22,23),(23,24),(24,25),
                             
                             (26,27),(27,28),(28,29),(29,30),
                             (26,31),(31,32),(32,33),(33,34),
                             (26,35),(35,36),(36,37),(37,38),
                             (26,39),(39,40),(40,41),(41,42),
                             (26,43),(43,44),(44,45),(45,46),
                            ]
            self.edge = self_link + neighbor_link
            self.center = 0
        else:
            raise ValueError('This layout is not supported!')

    def get_adjacency(self, strategy):
        valid_hop = range(0, self.max_hop + 1, self.dilation)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis ==
                                                                hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
            #self.A = np.swapaxes(np.swapaxes(A, 0, 1), 1, 2)
        else:
            raise ValueError("This strategy is not supported!")


def get_hop_distance(num_node, edge, max_hop=1):
    A = np.zeros((num_node, num_node))
    for i, j in edge:
        A[j, i] = 1
        A[i, j] = 1

    # compute hop steps
    hop_dis = np.zeros((num_node, num_node)) + np.inf
    transfer_mat = [np.linalg.matrix_power(A, d) for d in range(max_hop + 1)]
    arrive_mat = (np.stack(transfer_mat) > 0)
    for d in range(max_hop, -1, -1):
        hop_dis[arrive_mat[d]] = d
    return hop_dis


def normalize_digraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    AD = np.dot(A, Dn)
    return AD


def normalize_undigraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-0.5)
    DAD = np.dot(np.dot(Dn, A), Dn)
    return DAD

In [29]:
### Reference from: https://github.com/yysijie/st-gcn/tree/master/net

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# from Actionsrecognition.Utils import Graph


class GraphConvolution(nn.Module):
    """The basic module for applying a graph convolution.
    Args:
        - in_channel: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (int) Size of the graph convolving kernel.
        - t_kernel_size: (int) Size of the temporal convolving kernel.
        - t_stride: (int, optional) Stride of the temporal convolution. Default: 1
        - t_padding: (int, optional) Temporal zero-padding added to both sides of
            the input. Default: 0
        - t_dilation: (int, optional) Spacing between temporal kernel elements. Default: 1
        - bias: (bool, optional) If `True`, adds a learnable bias to the output.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math:`(N, in_channels, T_{in}, V)`,
                 A: Graph adjacency matrix in :math:`(K, V, V)`,
        - Output: Graph sequence out in :math:`(N, out_channels, T_{out}, V)`

            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.

    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 t_kernel_size=1,
                 t_stride=1,
                 t_padding=0,
                 t_dilation=1,
                 bias=True):
        super().__init__()

        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels,
                              out_channels * kernel_size,
                              kernel_size=(t_kernel_size, 1),
                              padding=(t_padding, 0),
                              stride=(t_stride, 1),
                              dilation=(t_dilation, 1),
                              bias=bias)

    def forward(self, x, A):
        x = self.conv(x)
        n, kc, t, v = x.size()
        x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
        x = torch.einsum('nkctv,kvw->nctw', (x, A))

        return x.contiguous()


class st_gcn(nn.Module):
    """Applies a spatial temporal graph convolution over an input graph sequence.
    Args:
        - in_channels: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (tuple) Size of the temporal convolving kernel and
            graph convolving kernel.
        - stride: (int, optional) Stride of the temporal convolution. Default: 1
        - dropout: (int, optional) Dropout rate of the final output. Default: 0
        - residual: (bool, optional) If `True`, applies a residual mechanism.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math: `(N, in_channels, T_{in}, V)`,
                 A: Graph Adjecency matrix in :math: `(K, V, V)`,
        - Output: Graph sequence out in :math: `(N, out_channels, T_{out}, V)`
            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.
    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1,
                 dropout=0,
                 residual=True):
        super().__init__()
        assert len(kernel_size) == 2
        assert kernel_size[0] % 2 == 1
        #print(kernel_size)(9, 3)
        padding = ((kernel_size[0] - 1) // 2, 0)
        #print(padding)(4, 0)

        self.gcn = GraphConvolution(in_channels, out_channels, kernel_size[1])
        self.tcn = nn.Sequential(nn.BatchNorm2d(out_channels),
                                 nn.ReLU(inplace=True),
                                 nn.Conv2d(out_channels,
                                           out_channels,
                                           (kernel_size[0], 1),
                                           (stride, 1),
                                           padding),
                                 nn.BatchNorm2d(out_channels),
                                 nn.Dropout(dropout, inplace=True),
                                 )

        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and (stride == 1):
            self.residual = lambda x: x
        else:
            self.residual = nn.Sequential(nn.Conv2d(in_channels,
                                                    out_channels,
                                                    kernel_size=1,
                                                    stride=(stride, 1)),
                                          nn.BatchNorm2d(out_channels)
                                          )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, A):
        res = self.residual(x)
        # print(res)
        x = self.gcn(x, A)
        #print("x_in:",x.size())
        x = self.tcn(x) + res
        #print("x_out:",x.size())
        return self.relu(x)


class StreamSpatialTemporalGraph(nn.Module):
    """Spatial temporal graph convolutional networks.
    Args:
        - in_channels: (int) Number of input channels.
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs. If `None` return pooling features of
            the last st-gcn layer instead.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :math:`(N, in_channels, T_{in}, V_{in})`
        - Output: :math:`(N, num_class)` where
            :math:`N` is a batch size,
            :math:`T_{in}` is a length of input sequence,
            :math:`V_{in}` is the number of graph nodes,
        or If num_class is `None`: `(N, out_channels)`
            :math:`out_channels` is number of out_channels of the last layer.
    """
    def __init__(self, in_channels, graph_args, num_class=None,
                 edge_importance_weighting=True, **kwargs):
        super().__init__()
        # Load graph.
        graph = Graph(**graph_args)
        A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
        self.register_buffer('A', A)

        # Networks.
        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 9
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        kwargs0 = {k: v for k, v in kwargs.items() if k != 'dropout'}

        self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
        self.st_gcn_networks = nn.ModuleList((
            st_gcn(in_channels, 64, kernel_size, 1, residual=False, **kwargs0),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 128, kernel_size, 2, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 256, kernel_size, 2, **kwargs),
            st_gcn(256, 256, kernel_size, 1, **kwargs),
            st_gcn(256, 256, kernel_size, 1, **kwargs)
        ))

        # initialize parameters for edge importance weighting.
        if edge_importance_weighting:
            self.edge_importance = nn.ParameterList([
                nn.Parameter(torch.ones(A.size()))
                for i in self.st_gcn_networks
            ])
        else:
            self.edge_importance = [1] * len(self.st_gcn_networks)

        if num_class is not None:
            self.cls = nn.Conv2d(256, num_class, kernel_size=1)
        else:
            self.cls = lambda x: x

    def forward(self, x):
        # data normalization.
        N, C, T, V = x.size()
        x = x.permute(0, 3, 1, 2).contiguous()  # (N, V, C, T)
        x = x.view(N, V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(N, C, T, V)

        # forward.
        for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
            x = gcn(x, self.A * importance)

        x = F.avg_pool2d(x, x.size()[2:])
        x = self.cls(x)
        x = x.view(x.size(0), -1)

        return x


class TwoStreamSpatialTemporalGraph(nn.Module):
    """Two inputs spatial temporal graph convolutional networks.
    Args:
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :tuple of math:`((N, 3, T, V), (N, 2, T, V))`
        for points and motions stream where.
            :math:`N` is a batch size,
            :math:`in_channels` is data channels (3 is (x, y, score)), (2 is (mot_x, mot_y))
            :math:`T` is a length of input sequence,
            :math:`V` is the number of graph nodes,
        - Output: :math:`(N, num_class)`
    """
    def __init__(self, graph_args, num_class, edge_importance_weighting=True,
                 **kwargs):
        super().__init__()
        self.pts_stream = StreamSpatialTemporalGraph(3, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)
        self.mot_stream = StreamSpatialTemporalGraph(3, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)

        self.fcn = nn.Linear(256 * 2, num_class)
        # self.fcn = nn.Linear(256 , num_class)
        
        # self.atten1 = nn.Linear(256 * 2, 128)
        # self.atten_bn = nn.BatchNorm1d(128)
        # self.atten_relu= nn.ReLU(inplace=True)
        # self.atten2 = nn.Linear(128,32)
        # self.atten_relu2= nn.ReLU(inplace=True)
        # self.atten3 = nn.Linear(32, 256 * 2)
        # self.atten_act = nn.Sigmoid() 
        
    def forward(self, inputs):
        out1 = self.pts_stream(inputs[0])
        out2 = self.mot_stream(inputs[1])
        
        #print(out1.size())torch.Size([32, 256])
        #print(out2.size())torch.Size([32, 256])
        concat = torch.cat([out1, out2], dim=-1)
        
        # attn = self.atten1(concat)
        # attn = self.atten_bn(attn)
        # attn = self.atten_relu(attn)
        # attn = self.atten2(attn)
        # attn = self.atten_relu2(attn)
        # attn = self.atten3(attn)
        # attn = self.atten_act(attn)
        # concat = concat * attn
        
        
        out = self.fcn(concat)
        # out = self.fcn(out1)
        
        return out
        # return torch.sigmoid(out)
        #return F.softmax(out,dim=-1)

In [30]:
import os
import time
import torch
import pickle
import numpy as np
import torch.nn.functional as F
from shutil import copyfile
from tqdm import tqdm
from torch.utils import data
from torch.optim.adadelta import Adadelta
from sklearn.model_selection import train_test_split

#device = 'cuda'
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("using", device, "device")

epochs = 30
batch_size = 32 #32

def load_dataset(data_files, batch_size, split_size=0.2,used_key_points=None):#0.2
    """Load data files into torch DataLoader with/without spliting train-test.
    """
    features, labels = [], []
    for fil in data_files:
        with open(fil, 'rb') as f:
            fts, lbs = pickle.load(f)
            features.append(fts)
            labels.append(lbs)
        del fts, lbs
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    if used_key_points != None:
        features = features[:,:,:,used_key_points]
    print(features.shape)
    print(labels.shape)

    if split_size > 0:
        x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=split_size,random_state=0,stratify=labels)
        
        train_set = data.TensorDataset(torch.tensor(x_train, dtype=torch.float32),torch.tensor(y_train, dtype=torch.int64))
        valid_set = data.TensorDataset(torch.tensor(x_valid, dtype=torch.float32),torch.tensor(y_valid, dtype=torch.int64))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = data.DataLoader(valid_set, batch_size)
    else:
        train_set = data.TensorDataset(torch.tensor(features, dtype=torch.float32),torch.tensor(labels, dtype=torch.int64))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = None
    return train_loader, valid_loader

def accuracy_batch(y_pred, y_true):
    # print(y_pred.shape,y_true.shape)
    # return (y_pred.argmax(1) == y_true.argmax(1)).mean()
    return (y_pred.argmax(1) == y_true).mean()


def set_training(model, mode=True):
    for p in model.parameters():
        p.requires_grad = mode
    model.train(mode)
    return model

save_folder = os.path.join(os.environ['HOME'],"KSL_V2/Outputs")
os.makedirs(save_folder,exist_ok=True)
used_key_points=[0,11,12,13,14]+[i for i in range(33,33+21)] + [i for i in range(54,54+21)] 
train_loader, valid_loader = load_dataset([os.path.join(os.environ['HOME'],"KSL_V2/Datasets/KSL_116_dataset.pkl")], 32,0.2,used_key_points) #batch_size = 32
dataloader = {'train': train_loader, 'valid': valid_loader}
num_class=20

graph_args = {'layout':'mediapipe_KSL','strategy': 'spatial'}
model = TwoStreamSpatialTemporalGraph(graph_args, num_class).to(device)
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"number of params: {n_parameters}")
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
losser = torch.nn.CrossEntropyLoss()

loss_list = {'train': [], 'valid': []}
accu_list = {'train': [], 'valid': []}
best_acc = -1
for e in range(epochs):
    print('Epoch {}/{}'.format(e, epochs - 1))
    for phase in ['train', 'valid']:
        if phase == 'train':
            model = set_training(model, True)
        else:
            model = set_training(model, False)

        run_loss = 0.0
        run_accu = 0.0
        with tqdm(dataloader[phase], desc=phase) as iterator:
            for pts, lbs in iterator:
                # Create motion input by distance of points (x, y) of the same node
                # in two frames.
                mot = pts[:, :, 1:, :] - pts[:, :, :-1, :]

                mot = mot.to(device)
                pts = pts.to(device)
                lbs = lbs.to(device)
                
                # Forward.
                out = model((pts, mot))
                #print(lbs)

                #print(out)
                loss = losser(out, lbs)

                if phase == 'train':
                    # Backward.
                    model.zero_grad()
                    loss.backward()
                    optimizer.step()

                run_loss += loss.item()
                accu = accuracy_batch(out.detach().cpu().numpy(),
                                      lbs.detach().cpu().numpy())
                run_accu += accu

                iterator.set_postfix_str(' loss: {:.4f}, accu: {:.4f}'.format(
                    loss.item(), accu))
                iterator.update()
                #break
        loss_list[phase].append(run_loss / len(iterator))
        accu_list[phase].append(run_accu / len(iterator))
        #print(accu_list)
        #print(torch.max(accu_list))
    if(best_acc < accu_list['valid'][-1]):
        best_acc = accu_list['valid'][-1]
        torch.save(model.state_dict(), os.path.join(save_folder, 'tsstg-model_best.pth'))
        #break

    print('Summary epoch:\n - Train loss: {:.4f}, accu: {:.4f}\n - Valid loss:'
          ' {:.4f}, accu: {:.4f}'.format(loss_list['train'][-1], accu_list['train'][-1],
                                         loss_list['valid'][-1], accu_list['valid'][-1]))
del model

using cuda device
(17983, 3, 116, 47)
(17983,)
number of params: 6272388
Epoch 0/29


valid: 100%|██████████| 113/113 [00:08<00:00, 12.66it/s,  loss: 0.9989, accu: 0.5385]


Summary epoch:
 - Train loss: 1.1090, accu: 0.6188
 - Valid loss: 0.7139, accu: 0.7365
Epoch 1/29


valid: 100%|██████████| 113/113 [00:08<00:00, 12.60it/s,  loss: 1.4659, accu: 0.6923]


Summary epoch:
 - Train loss: 0.3621, accu: 0.8714
 - Valid loss: 1.2937, accu: 0.6690
Epoch 2/29


...

Summary epoch:
 - Train loss: 0.0099, accu: 0.9972
 - Valid loss: 0.0026, accu: 0.9986
Epoch 28/29


valid: 100%|██████████| 113/113 [00:08<00:00, 12.63it/s,  loss: 0.0000, accu: 1.0000]


Summary epoch:
 - Train loss: 0.0057, accu: 0.9978
 - Valid loss: 0.0007, accu: 1.0000
Epoch 29/29


valid: 100%|██████████| 113/113 [00:08<00:00, 12.60it/s,  loss: 0.0007, accu: 1.0000]

Summary epoch:
 - Train loss: 0.0065, accu: 0.9978
 - Valid loss: 0.0020, accu: 0.9989


# KSL77 Dataset ImageClassification

In [1]:
### Reference from: https://github.com/yysijie/st-gcn/blob/master/net/utils/graph.py

import os
import torch
import numpy as np


class Graph:
    """The Graph to model the skeletons extracted by the Alpha-Pose.
    Args:
        - strategy: (string) must be one of the follow candidates
            - uniform: Uniform Labeling,
            - distance: Distance Partitioning,
            - spatial: Spatial Configuration,
        For more information, please refer to the section 'Partition Strategies'
            in our paper (https://arxiv.org/abs/1801.07455).
        - layout: (string) must be one of the follow candidates
            - coco_cut: Is COCO format but cut 4 joints (L-R ears, L-R eyes) out.
        - max_hop: (int) the maximal distance between two connected nodes.
        - dilation: (int) controls the spacing between the kernel points.
    """
    def __init__(self,
                 layout='coco_cut',
                 strategy='uniform',
                 max_hop=1,
                 dilation=1):
        self.max_hop = max_hop
        self.dilation = dilation

        self.get_edge(layout)
        self.hop_dis = get_hop_distance(self.num_node, self.edge, max_hop)
        self.get_adjacency(strategy)

    def get_edge(self, layout):
        if layout == 'coco_cut':
            self.num_node = 14
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(6, 4), (4, 2), (2, 13), (13, 1), (5, 3), (3, 1), (12, 10),
                             (10, 8), (8, 2), (11, 9), (9, 7), (7, 1), (13, 0)]
            self.edge = self_link + neighbor_link
            self.center = 13
            
        elif layout == 'mediapipe_KSL':
            self.num_node = 47
            self_link = [(i, i) for i in range(self.num_node)]
            # used_key_points=
            # [0,11,12,13,14]+[i for i in range(33,33+21)] + [i for i in range(54,54+21)] 
            neighbor_link = [(0,1),(0,2),(1,3),(2,4),(3,26),(4,5), # nose-arms-wrist
                             
                             (5,6),(6,7),(7,8),(8,9),
                             (5,10),(10,11),(11,12),(12,13),
                             (5,14),(14,15),(15,16),(16,17),
                             (5,18),(18,19),(19,20),(20,21),
                             (5,22),(22,23),(23,24),(24,25),
                             
                             (26,27),(27,28),(28,29),(29,30),
                             (26,31),(31,32),(32,33),(33,34),
                             (26,35),(35,36),(36,37),(37,38),
                             (26,39),(39,40),(40,41),(41,42),
                             (26,43),(43,44),(44,45),(45,46),
                            ]
            self.edge = self_link + neighbor_link
            self.center = 0
        else:
            raise ValueError('This layout is not supported!')

    def get_adjacency(self, strategy):
        valid_hop = range(0, self.max_hop + 1, self.dilation)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis ==
                                                                hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
            #self.A = np.swapaxes(np.swapaxes(A, 0, 1), 1, 2)
        else:
            raise ValueError("This strategy is not supported!")


def get_hop_distance(num_node, edge, max_hop=1):
    A = np.zeros((num_node, num_node))
    for i, j in edge:
        A[j, i] = 1
        A[i, j] = 1

    # compute hop steps
    hop_dis = np.zeros((num_node, num_node)) + np.inf
    transfer_mat = [np.linalg.matrix_power(A, d) for d in range(max_hop + 1)]
    arrive_mat = (np.stack(transfer_mat) > 0)
    for d in range(max_hop, -1, -1):
        hop_dis[arrive_mat[d]] = d
    return hop_dis


def normalize_digraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    AD = np.dot(A, Dn)
    return AD


def normalize_undigraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-0.5)
    DAD = np.dot(np.dot(Dn, A), Dn)
    return DAD

In [2]:
### Reference from: https://github.com/yysijie/st-gcn/tree/master/net

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# from Actionsrecognition.Utils import Graph


class GraphConvolution(nn.Module):
    """The basic module for applying a graph convolution.
    Args:
        - in_channel: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (int) Size of the graph convolving kernel.
        - t_kernel_size: (int) Size of the temporal convolving kernel.
        - t_stride: (int, optional) Stride of the temporal convolution. Default: 1
        - t_padding: (int, optional) Temporal zero-padding added to both sides of
            the input. Default: 0
        - t_dilation: (int, optional) Spacing between temporal kernel elements. Default: 1
        - bias: (bool, optional) If `True`, adds a learnable bias to the output.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math:`(N, in_channels, T_{in}, V)`,
                 A: Graph adjacency matrix in :math:`(K, V, V)`,
        - Output: Graph sequence out in :math:`(N, out_channels, T_{out}, V)`

            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.

    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 t_kernel_size=1,
                 t_stride=1,
                 t_padding=0,
                 t_dilation=1,
                 bias=True):
        super().__init__()

        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels,
                              out_channels * kernel_size,
                              kernel_size=(t_kernel_size, 1),
                              padding=(t_padding, 0),
                              stride=(t_stride, 1),
                              dilation=(t_dilation, 1),
                              bias=bias)

    def forward(self, x, A):
        x = self.conv(x)
        n, kc, t, v = x.size()
        x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
        x = torch.einsum('nkctv,kvw->nctw', (x, A))

        return x.contiguous()


class st_gcn(nn.Module):
    """Applies a spatial temporal graph convolution over an input graph sequence.
    Args:
        - in_channels: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (tuple) Size of the temporal convolving kernel and
            graph convolving kernel.
        - stride: (int, optional) Stride of the temporal convolution. Default: 1
        - dropout: (int, optional) Dropout rate of the final output. Default: 0
        - residual: (bool, optional) If `True`, applies a residual mechanism.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math: `(N, in_channels, T_{in}, V)`,
                 A: Graph Adjecency matrix in :math: `(K, V, V)`,
        - Output: Graph sequence out in :math: `(N, out_channels, T_{out}, V)`
            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.
    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1,
                 dropout=0,
                 residual=True):
        super().__init__()
        assert len(kernel_size) == 2
        assert kernel_size[0] % 2 == 1
        #print(kernel_size)(9, 3)
        padding = ((kernel_size[0] - 1) // 2, 0)
        #print(padding)(4, 0)

        self.gcn = GraphConvolution(in_channels, out_channels, kernel_size[1])
        self.tcn = nn.Sequential(nn.BatchNorm2d(out_channels),
                                 nn.ReLU(inplace=True),
                                 nn.Conv2d(out_channels,
                                           out_channels,
                                           (kernel_size[0], 1),
                                           (stride, 1),
                                           padding),
                                 nn.BatchNorm2d(out_channels),
                                 nn.Dropout(dropout, inplace=True),
                                 )

        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and (stride == 1):
            self.residual = lambda x: x
        else:
            self.residual = nn.Sequential(nn.Conv2d(in_channels,
                                                    out_channels,
                                                    kernel_size=1,
                                                    stride=(stride, 1)),
                                          nn.BatchNorm2d(out_channels)
                                          )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, A):
        res = self.residual(x)
        # print(res)
        x = self.gcn(x, A)
        #print("x_in:",x.size())
        x = self.tcn(x) + res
        #print("x_out:",x.size())
        return self.relu(x)


class StreamSpatialTemporalGraph(nn.Module):
    """Spatial temporal graph convolutional networks.
    Args:
        - in_channels: (int) Number of input channels.
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs. If `None` return pooling features of
            the last st-gcn layer instead.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :math:`(N, in_channels, T_{in}, V_{in})`
        - Output: :math:`(N, num_class)` where
            :math:`N` is a batch size,
            :math:`T_{in}` is a length of input sequence,
            :math:`V_{in}` is the number of graph nodes,
        or If num_class is `None`: `(N, out_channels)`
            :math:`out_channels` is number of out_channels of the last layer.
    """
    def __init__(self, in_channels, graph_args, num_class=None,
                 edge_importance_weighting=True, **kwargs):
        super().__init__()
        # Load graph.
        graph = Graph(**graph_args)
        A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
        self.register_buffer('A', A)

        # Networks.
        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 9
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        kwargs0 = {k: v for k, v in kwargs.items() if k != 'dropout'}

        self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
        self.st_gcn_networks = nn.ModuleList((
            st_gcn(in_channels, 64, kernel_size, 1, residual=False, **kwargs0),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 128, kernel_size, 2, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 256, kernel_size, 2, **kwargs),
            st_gcn(256, 256, kernel_size, 1, **kwargs),
            st_gcn(256, 256, kernel_size, 1, **kwargs)
        ))

        # initialize parameters for edge importance weighting.
        if edge_importance_weighting:
            self.edge_importance = nn.ParameterList([
                nn.Parameter(torch.ones(A.size()))
                for i in self.st_gcn_networks
            ])
        else:
            self.edge_importance = [1] * len(self.st_gcn_networks)

        if num_class is not None:
            self.cls = nn.Conv2d(256, num_class, kernel_size=1)
        else:
            self.cls = lambda x: x

    def forward(self, x):
        # data normalization.
        N, C, T, V = x.size()
        x = x.permute(0, 3, 1, 2).contiguous()  # (N, V, C, T)
        x = x.view(N, V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(N, C, T, V)

        # forward.
        for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
            x = gcn(x, self.A * importance)

        x = F.avg_pool2d(x, x.size()[2:])
        x = self.cls(x)
        x = x.view(x.size(0), -1)

        return x


class TwoStreamSpatialTemporalGraph(nn.Module):
    """Two inputs spatial temporal graph convolutional networks.
    Args:
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :tuple of math:`((N, 3, T, V), (N, 2, T, V))`
        for points and motions stream where.
            :math:`N` is a batch size,
            :math:`in_channels` is data channels (3 is (x, y, score)), (2 is (mot_x, mot_y))
            :math:`T` is a length of input sequence,
            :math:`V` is the number of graph nodes,
        - Output: :math:`(N, num_class)`
    """
    def __init__(self, graph_args, num_class, edge_importance_weighting=True,
                 **kwargs):
        super().__init__()
        self.pts_stream = StreamSpatialTemporalGraph(3, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)
        self.mot_stream = StreamSpatialTemporalGraph(3, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)

        # self.fcn = nn.Linear(256 * 2, num_class)
        self.fcn = nn.Linear(256 , num_class)
        
        # self.atten1 = nn.Linear(256 * 2, 128)
        # self.atten_bn = nn.BatchNorm1d(128)
        # self.atten_relu= nn.ReLU(inplace=True)
        # self.atten2 = nn.Linear(128,32)
        # self.atten_relu2= nn.ReLU(inplace=True)
        # self.atten3 = nn.Linear(32, 256 * 2)
        # self.atten_act = nn.Sigmoid() 
        
    def forward(self, inputs):
        out1 = self.pts_stream(inputs[0])
        # out2 = self.mot_stream(inputs[1])
        
        #print(out1.size())torch.Size([32, 256])
        #print(out2.size())torch.Size([32, 256])
        # concat = torch.cat([out1, out2], dim=-1)
        
        # attn = self.atten1(concat)
        # attn = self.atten_bn(attn)
        # attn = self.atten_relu(attn)
        # attn = self.atten2(attn)
        # attn = self.atten_relu2(attn)
        # attn = self.atten3(attn)
        # attn = self.atten_act(attn)
        # concat = concat * attn
        
        
        # out = self.fcn(concat)
        out = self.fcn(out1)
        
        return out
        # return torch.sigmoid(out)
        #return F.softmax(out,dim=-1)

In [3]:
import os
import time
import torch
import pickle
import numpy as np
import torch.nn.functional as F
from shutil import copyfile
from tqdm import tqdm
from torch.utils import data
from torch.optim.adadelta import Adadelta
from sklearn.model_selection import train_test_split

#device = 'cuda'
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("using", device, "device")

epochs = 100
batch_size = 64 #32

def load_dataset(data_files, batch_size, split_size=0.2,used_key_points=None):#0.2
    """Load data files into torch DataLoader with/without spliting train-test.
    """
    features, labels = [], []
    for fil in data_files:
        with open(fil, 'rb') as f:
            fts, lbs = pickle.load(f)
            features.append(fts)
            labels.append(lbs)
        del fts, lbs
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    if used_key_points != None:
        features = features[:,:,:,used_key_points]
    print(features.shape)
    print(labels.shape)

    if split_size > 0:
        x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=split_size,random_state=0,stratify=labels)
        
        train_set = data.TensorDataset(torch.tensor(x_train, dtype=torch.float32),torch.tensor(y_train, dtype=torch.int64))
        valid_set = data.TensorDataset(torch.tensor(x_valid, dtype=torch.float32),torch.tensor(y_valid, dtype=torch.int64))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = data.DataLoader(valid_set, batch_size)
    else:
        train_set = data.TensorDataset(torch.tensor(features, dtype=torch.float32),torch.tensor(labels, dtype=torch.int64))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = None
    return train_loader, valid_loader

def accuracy_batch(y_pred, y_true):
    # print(y_pred.shape,y_true.shape)
    # return (y_pred.argmax(1) == y_true.argmax(1)).mean()
    return (y_pred.argmax(1) == y_true).mean()


def set_training(model, mode=True):
    for p in model.parameters():
        p.requires_grad = mode
    model.train(mode)
    return model

save_folder = os.path.join(os.environ['HOME'],"KSL_V2/Outputs")
os.makedirs(save_folder,exist_ok=True)
used_key_points=[0,11,12,13,14]+[i for i in range(33,33+21)] + [i for i in range(54,54+21)] 
train_loader, valid_loader = load_dataset([os.path.join(os.environ['HOME'],"KSL_V2/Datasets/KSL77_1_dataset.pkl")], 32,0.2,used_key_points) #batch_size = 32
dataloader = {'train': train_loader, 'valid': valid_loader}
num_class=77

graph_args = {'layout':'mediapipe_KSL','strategy': 'spatial'}
model = TwoStreamSpatialTemporalGraph(graph_args, num_class).to(device)
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"number of params: {n_parameters}")
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0001)
losser = torch.nn.CrossEntropyLoss()

loss_list = {'train': [], 'valid': []}
accu_list = {'train': [], 'valid': []}
best_acc = -1
for e in range(epochs):
    print('Epoch {}/{}'.format(e, epochs - 1))
    for phase in ['train', 'valid']:
        if phase == 'train':
            model = set_training(model, True)
        else:
            model = set_training(model, False)

        run_loss = 0.0
        run_accu = 0.0
        with tqdm(dataloader[phase], desc=phase) as iterator:
            for pts, lbs in iterator:
                # Create motion input by distance of points (x, y) of the same node
                # in two frames.
                mot = pts[:, :, 1:, :] - pts[:, :, :-1, :]

                mot = mot.to(device)
                pts = pts.to(device)
                lbs = lbs.to(device)
                
                # Forward.
                out = model((pts, mot))
                #print(lbs)

                #print(out)
                loss = losser(out, lbs)

                if phase == 'train':
                    # Backward.
                    model.zero_grad()
                    loss.backward()
                    optimizer.step()

                run_loss += loss.item()
                accu = accuracy_batch(out.detach().cpu().numpy(),
                                      lbs.detach().cpu().numpy())
                run_accu += accu

                iterator.set_postfix_str(' loss: {:.4f}, accu: {:.4f}'.format(
                    loss.item(), accu))
                iterator.update()
                #break
        loss_list[phase].append(run_loss / len(iterator))
        accu_list[phase].append(run_accu / len(iterator))
        #print(accu_list)
        #print(torch.max(accu_list))
    if(best_acc < accu_list['valid'][-1]):
        best_acc = accu_list['valid'][-1]
        torch.save(model.state_dict(), os.path.join(save_folder, 'tsstg-model_best.pth'))
        #break

    print('Summary epoch:\n - Train loss: {:.4f}, accu: {:.4f}\n - Valid loss:'
          ' {:.4f}, accu: {:.4f}'.format(loss_list['train'][-1], accu_list['train'][-1],
                                         loss_list['valid'][-1], accu_list['valid'][-1]))
del model

using cuda device
(108373, 3, 1, 47)
(108373,)
number of params: 6281917
Epoch 0/99


valid: 100%|██████████| 678/678 [00:03<00:00, 190.58it/s,  loss: 1.9834, accu: 0.3636]


Summary epoch:
 - Train loss: 2.9000, accu: 0.2425
 - Valid loss: 2.3800, accu: 0.3538
Epoch 1/99


valid: 100%|██████████| 678/678 [00:03<00:00, 191.18it/s,  loss: 1.7863, accu: 0.4545]


Summary epoch:
 - Train loss: 2.1478, accu: 0.4137
 - Valid loss: 2.1252, accu: 0.4200
Epoch 2/99


...

Summary epoch:
 - Train loss: 0.1795, accu: 0.9420
 - Valid loss: 0.1534, accu: 0.9573
Epoch 98/99


valid: 100%|██████████| 678/678 [00:03<00:00, 222.60it/s,  loss: 0.0106, accu: 1.0000]


Summary epoch:
 - Train loss: 0.1914, accu: 0.9387
 - Valid loss: 0.1898, accu: 0.9502
Epoch 99/99


valid: 100%|██████████| 678/678 [00:03<00:00, 222.87it/s,  loss: 0.8458, accu: 0.7273]

Summary epoch:
 - Train loss: 0.1830, accu: 0.9406
 - Valid loss: 0.5291, accu: 0.8542


# KSL77 Dataset Video Classification

In [4]:
### Reference from: https://github.com/yysijie/st-gcn/blob/master/net/utils/graph.py

import os
import torch
import numpy as np


class Graph:
    """The Graph to model the skeletons extracted by the Alpha-Pose.
    Args:
        - strategy: (string) must be one of the follow candidates
            - uniform: Uniform Labeling,
            - distance: Distance Partitioning,
            - spatial: Spatial Configuration,
        For more information, please refer to the section 'Partition Strategies'
            in our paper (https://arxiv.org/abs/1801.07455).
        - layout: (string) must be one of the follow candidates
            - coco_cut: Is COCO format but cut 4 joints (L-R ears, L-R eyes) out.
        - max_hop: (int) the maximal distance between two connected nodes.
        - dilation: (int) controls the spacing between the kernel points.
    """
    def __init__(self,
                 layout='coco_cut',
                 strategy='uniform',
                 max_hop=1,
                 dilation=1):
        self.max_hop = max_hop
        self.dilation = dilation

        self.get_edge(layout)
        self.hop_dis = get_hop_distance(self.num_node, self.edge, max_hop)
        self.get_adjacency(strategy)

    def get_edge(self, layout):
        if layout == 'coco_cut':
            self.num_node = 14
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(6, 4), (4, 2), (2, 13), (13, 1), (5, 3), (3, 1), (12, 10),
                             (10, 8), (8, 2), (11, 9), (9, 7), (7, 1), (13, 0)]
            self.edge = self_link + neighbor_link
            self.center = 13
            
        elif layout == 'mediapipe_KSL':
            self.num_node = 47
            self_link = [(i, i) for i in range(self.num_node)]
            # used_key_points=
            # [0,11,12,13,14]+[i for i in range(33,33+21)] + [i for i in range(54,54+21)] 
            neighbor_link = [(0,1),(0,2),(1,3),(2,4),(3,26),(4,5), # nose-arms-wrist
                             
                             (5,6),(6,7),(7,8),(8,9),
                             (5,10),(10,11),(11,12),(12,13),
                             (5,14),(14,15),(15,16),(16,17),
                             (5,18),(18,19),(19,20),(20,21),
                             (5,22),(22,23),(23,24),(24,25),
                             
                             (26,27),(27,28),(28,29),(29,30),
                             (26,31),(31,32),(32,33),(33,34),
                             (26,35),(35,36),(36,37),(37,38),
                             (26,39),(39,40),(40,41),(41,42),
                             (26,43),(43,44),(44,45),(45,46),
                            ]
            self.edge = self_link + neighbor_link
            self.center = 0
        else:
            raise ValueError('This layout is not supported!')

    def get_adjacency(self, strategy):
        valid_hop = range(0, self.max_hop + 1, self.dilation)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis ==
                                                                hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
            #self.A = np.swapaxes(np.swapaxes(A, 0, 1), 1, 2)
        else:
            raise ValueError("This strategy is not supported!")


def get_hop_distance(num_node, edge, max_hop=1):
    A = np.zeros((num_node, num_node))
    for i, j in edge:
        A[j, i] = 1
        A[i, j] = 1

    # compute hop steps
    hop_dis = np.zeros((num_node, num_node)) + np.inf
    transfer_mat = [np.linalg.matrix_power(A, d) for d in range(max_hop + 1)]
    arrive_mat = (np.stack(transfer_mat) > 0)
    for d in range(max_hop, -1, -1):
        hop_dis[arrive_mat[d]] = d
    return hop_dis


def normalize_digraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    AD = np.dot(A, Dn)
    return AD


def normalize_undigraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-0.5)
    DAD = np.dot(np.dot(Dn, A), Dn)
    return DAD

In [5]:
### Reference from: https://github.com/yysijie/st-gcn/tree/master/net

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# from Actionsrecognition.Utils import Graph


class GraphConvolution(nn.Module):
    """The basic module for applying a graph convolution.
    Args:
        - in_channel: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (int) Size of the graph convolving kernel.
        - t_kernel_size: (int) Size of the temporal convolving kernel.
        - t_stride: (int, optional) Stride of the temporal convolution. Default: 1
        - t_padding: (int, optional) Temporal zero-padding added to both sides of
            the input. Default: 0
        - t_dilation: (int, optional) Spacing between temporal kernel elements. Default: 1
        - bias: (bool, optional) If `True`, adds a learnable bias to the output.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math:`(N, in_channels, T_{in}, V)`,
                 A: Graph adjacency matrix in :math:`(K, V, V)`,
        - Output: Graph sequence out in :math:`(N, out_channels, T_{out}, V)`

            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.

    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 t_kernel_size=1,
                 t_stride=1,
                 t_padding=0,
                 t_dilation=1,
                 bias=True):
        super().__init__()

        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels,
                              out_channels * kernel_size,
                              kernel_size=(t_kernel_size, 1),
                              padding=(t_padding, 0),
                              stride=(t_stride, 1),
                              dilation=(t_dilation, 1),
                              bias=bias)

    def forward(self, x, A):
        x = self.conv(x)
        n, kc, t, v = x.size()
        x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
        x = torch.einsum('nkctv,kvw->nctw', (x, A))

        return x.contiguous()


class st_gcn(nn.Module):
    """Applies a spatial temporal graph convolution over an input graph sequence.
    Args:
        - in_channels: (int) Number of channels in the input sequence data.
        - out_channels: (int) Number of channels produced by the convolution.
        - kernel_size: (tuple) Size of the temporal convolving kernel and
            graph convolving kernel.
        - stride: (int, optional) Stride of the temporal convolution. Default: 1
        - dropout: (int, optional) Dropout rate of the final output. Default: 0
        - residual: (bool, optional) If `True`, applies a residual mechanism.
            Default: `True`
    Shape:
        - Inputs x: Graph sequence in :math: `(N, in_channels, T_{in}, V)`,
                 A: Graph Adjecency matrix in :math: `(K, V, V)`,
        - Output: Graph sequence out in :math: `(N, out_channels, T_{out}, V)`
            where
                :math:`N` is a batch size,
                :math:`K` is the spatial kernel size, as :math:`K == kernel_size[1]`,
                :math:`T_{in}/T_{out}` is a length of input/output sequence,
                :math:`V` is the number of graph nodes.
    """
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1,
                 dropout=0,
                 residual=True):
        super().__init__()
        assert len(kernel_size) == 2
        assert kernel_size[0] % 2 == 1
        #print(kernel_size)(9, 3)
        padding = ((kernel_size[0] - 1) // 2, 0)
        #print(padding)(4, 0)

        self.gcn = GraphConvolution(in_channels, out_channels, kernel_size[1])
        self.tcn = nn.Sequential(nn.BatchNorm2d(out_channels),
                                 nn.ReLU(inplace=True),
                                 nn.Conv2d(out_channels,
                                           out_channels,
                                           (kernel_size[0], 1),
                                           (stride, 1),
                                           padding),
                                 nn.BatchNorm2d(out_channels),
                                 nn.Dropout(dropout, inplace=True),
                                 )

        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and (stride == 1):
            self.residual = lambda x: x
        else:
            self.residual = nn.Sequential(nn.Conv2d(in_channels,
                                                    out_channels,
                                                    kernel_size=1,
                                                    stride=(stride, 1)),
                                          nn.BatchNorm2d(out_channels)
                                          )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, A):
        res = self.residual(x)
        # print(res)
        x = self.gcn(x, A)
        #print("x_in:",x.size())
        x = self.tcn(x) + res
        #print("x_out:",x.size())
        return self.relu(x)


class StreamSpatialTemporalGraph(nn.Module):
    """Spatial temporal graph convolutional networks.
    Args:
        - in_channels: (int) Number of input channels.
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs. If `None` return pooling features of
            the last st-gcn layer instead.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :math:`(N, in_channels, T_{in}, V_{in})`
        - Output: :math:`(N, num_class)` where
            :math:`N` is a batch size,
            :math:`T_{in}` is a length of input sequence,
            :math:`V_{in}` is the number of graph nodes,
        or If num_class is `None`: `(N, out_channels)`
            :math:`out_channels` is number of out_channels of the last layer.
    """
    def __init__(self, in_channels, graph_args, num_class=None,
                 edge_importance_weighting=True, **kwargs):
        super().__init__()
        # Load graph.
        graph = Graph(**graph_args)
        A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
        self.register_buffer('A', A)

        # Networks.
        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 9
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        kwargs0 = {k: v for k, v in kwargs.items() if k != 'dropout'}

        self.data_bn = nn.BatchNorm1d(in_channels * A.size(1))
        self.st_gcn_networks = nn.ModuleList((
            st_gcn(in_channels, 64, kernel_size, 1, residual=False, **kwargs0),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 64, kernel_size, 1, **kwargs),
            st_gcn(64, 128, kernel_size, 2, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 128, kernel_size, 1, **kwargs),
            st_gcn(128, 256, kernel_size, 2, **kwargs),
            st_gcn(256, 256, kernel_size, 1, **kwargs),
            st_gcn(256, 256, kernel_size, 1, **kwargs)
        ))

        # initialize parameters for edge importance weighting.
        if edge_importance_weighting:
            self.edge_importance = nn.ParameterList([
                nn.Parameter(torch.ones(A.size()))
                for i in self.st_gcn_networks
            ])
        else:
            self.edge_importance = [1] * len(self.st_gcn_networks)

        if num_class is not None:
            self.cls = nn.Conv2d(256, num_class, kernel_size=1)
        else:
            self.cls = lambda x: x

    def forward(self, x):
        # data normalization.
        N, C, T, V = x.size()
        x = x.permute(0, 3, 1, 2).contiguous()  # (N, V, C, T)
        x = x.view(N, V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(N, C, T, V)

        # forward.
        for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
            x = gcn(x, self.A * importance)

        x = F.avg_pool2d(x, x.size()[2:])
        x = self.cls(x)
        x = x.view(x.size(0), -1)

        return x


class TwoStreamSpatialTemporalGraph(nn.Module):
    """Two inputs spatial temporal graph convolutional networks.
    Args:
        - graph_args: (dict) Args map of `Actionsrecognition.Utils.Graph` Class.
        - num_class: (int) Number of class outputs.
        - edge_importance_weighting: (bool) If `True`, adds a learnable importance
            weighting to the edges of the graph.
        - **kwargs: (optional) Other parameters for graph convolution units.
    Shape:
        - Input: :tuple of math:`((N, 3, T, V), (N, 2, T, V))`
        for points and motions stream where.
            :math:`N` is a batch size,
            :math:`in_channels` is data channels (3 is (x, y, score)), (2 is (mot_x, mot_y))
            :math:`T` is a length of input sequence,
            :math:`V` is the number of graph nodes,
        - Output: :math:`(N, num_class)`
    """
    def __init__(self, graph_args, num_class, edge_importance_weighting=True,
                 **kwargs):
        super().__init__()
        self.pts_stream = StreamSpatialTemporalGraph(3, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)
        self.mot_stream = StreamSpatialTemporalGraph(3, graph_args, None,
                                                     edge_importance_weighting,
                                                     **kwargs)

        self.fcn = nn.Linear(256 * 2, num_class)
        # self.fcn = nn.Linear(256 , num_class)
        
        # self.atten1 = nn.Linear(256 * 2, 128)
        # self.atten_bn = nn.BatchNorm1d(128)
        # self.atten_relu= nn.ReLU(inplace=True)
        # self.atten2 = nn.Linear(128,32)
        # self.atten_relu2= nn.ReLU(inplace=True)
        # self.atten3 = nn.Linear(32, 256 * 2)
        # self.atten_act = nn.Sigmoid() 
        
    def forward(self, inputs):
        out1 = self.pts_stream(inputs[0])
        out2 = self.mot_stream(inputs[1])
        
        #print(out1.size())torch.Size([32, 256])
        #print(out2.size())torch.Size([32, 256])
        concat = torch.cat([out1, out2], dim=-1)
        
        # attn = self.atten1(concat)
        # attn = self.atten_bn(attn)
        # attn = self.atten_relu(attn)
        # attn = self.atten2(attn)
        # attn = self.atten_relu2(attn)
        # attn = self.atten3(attn)
        # attn = self.atten_act(attn)
        # concat = concat * attn
        
        
        out = self.fcn(concat)
        # out = self.fcn(out1)
        
        return out
        # return torch.sigmoid(out)
        #return F.softmax(out,dim=-1)

In [6]:
import os
import time
import torch
import pickle
import numpy as np
import torch.nn.functional as F
from shutil import copyfile
from tqdm import tqdm
from torch.utils import data
from torch.optim.adadelta import Adadelta
from sklearn.model_selection import train_test_split

#device = 'cuda'
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("using", device, "device")

epochs = 100
batch_size = 64 #32

def load_dataset(data_files, batch_size, split_size=0.2,used_key_points=None):#0.2
    """Load data files into torch DataLoader with/without spliting train-test.
    """
    features, labels = [], []
    for fil in data_files:
        with open(fil, 'rb') as f:
            fts, lbs = pickle.load(f)
            features.append(fts)
            labels.append(lbs)
        del fts, lbs
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    if used_key_points != None:
        features = features[:,:,:,used_key_points]
    print(features.shape)
    print(labels.shape)

    if split_size > 0:
        x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=split_size,random_state=0,stratify=labels)
        
        train_set = data.TensorDataset(torch.tensor(x_train, dtype=torch.float32),torch.tensor(y_train, dtype=torch.int64))
        valid_set = data.TensorDataset(torch.tensor(x_valid, dtype=torch.float32),torch.tensor(y_valid, dtype=torch.int64))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = data.DataLoader(valid_set, batch_size)
    else:
        train_set = data.TensorDataset(torch.tensor(features, dtype=torch.float32),torch.tensor(labels, dtype=torch.int64))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = None
    return train_loader, valid_loader

def accuracy_batch(y_pred, y_true):
    # print(y_pred.shape,y_true.shape)
    # return (y_pred.argmax(1) == y_true.argmax(1)).mean()
    return (y_pred.argmax(1) == y_true).mean()


def set_training(model, mode=True):
    for p in model.parameters():
        p.requires_grad = mode
    model.train(mode)
    return model

save_folder = os.path.join(os.environ['HOME'],"KSL_V2/Outputs")
os.makedirs(save_folder,exist_ok=True)
used_key_points=[0,11,12,13,14]+[i for i in range(33,33+21)] + [i for i in range(54,54+21)] 
train_loader, valid_loader = load_dataset([os.path.join(os.environ['HOME'],"KSL_V2/Datasets/KSL77_25_dataset.pkl")], 32,0.2,used_key_points) #batch_size = 32
dataloader = {'train': train_loader, 'valid': valid_loader}
num_class=77

graph_args = {'layout':'mediapipe_KSL','strategy': 'spatial'}
model = TwoStreamSpatialTemporalGraph(graph_args, num_class).to(device)
n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"number of params: {n_parameters}")
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
losser = torch.nn.CrossEntropyLoss()

loss_list = {'train': [], 'valid': []}
accu_list = {'train': [], 'valid': []}
best_acc = -1
for e in range(epochs):
    print('Epoch {}/{}'.format(e, epochs - 1))
    for phase in ['train', 'valid']:
        if phase == 'train':
            model = set_training(model, True)
        else:
            model = set_training(model, False)

        run_loss = 0.0
        run_accu = 0.0
        with tqdm(dataloader[phase], desc=phase) as iterator:
            for pts, lbs in iterator:
                # Create motion input by distance of points (x, y) of the same node
                # in two frames.
                mot = pts[:, :, 1:, :] - pts[:, :, :-1, :]

                mot = mot.to(device)
                pts = pts.to(device)
                lbs = lbs.to(device)
                
                # Forward.
                out = model((pts, mot))
                #print(lbs)

                #print(out)
                loss = losser(out, lbs)

                if phase == 'train':
                    # Backward.
                    model.zero_grad()
                    loss.backward()
                    optimizer.step()

                run_loss += loss.item()
                accu = accuracy_batch(out.detach().cpu().numpy(),
                                      lbs.detach().cpu().numpy())
                run_accu += accu

                iterator.set_postfix_str(' loss: {:.4f}, accu: {:.4f}'.format(
                    loss.item(), accu))
                iterator.update()
                #break
        loss_list[phase].append(run_loss / len(iterator))
        accu_list[phase].append(run_accu / len(iterator))
        #print(accu_list)
        #print(torch.max(accu_list))
    if(best_acc < accu_list['valid'][-1]):
        best_acc = accu_list['valid'][-1]
        torch.save(model.state_dict(), os.path.join(save_folder, 'tsstg-model_best.pth'))
        #break

    print('Summary epoch:\n - Train loss: {:.4f}, accu: {:.4f}\n - Valid loss:'
          ' {:.4f}, accu: {:.4f}'.format(loss_list['train'][-1], accu_list['train'][-1],
                                         loss_list['valid'][-1], accu_list['valid'][-1]))
del model
#0.996577079107505

using cuda device
(78877, 3, 25, 47)
(78877,)
number of params: 6301629
Epoch 0/99


valid: 100%|██████████| 493/493 [00:09<00:00, 53.01it/s,  loss: 2.2034, accu: 0.3750]


Summary epoch:
 - Train loss: 2.5456, accu: 0.2990
 - Valid loss: 1.8127, accu: 0.4618
Epoch 1/99


valid: 100%|██████████| 493/493 [00:09<00:00, 52.69it/s,  loss: 1.8028, accu: 0.4688]


Summary epoch:
 - Train loss: 1.3821, accu: 0.5894
 - Valid loss: 1.4313, accu: 0.5927
Epoch 2/99


...

Summary epoch:
 - Train loss: 0.0087, accu: 0.9974
 - Valid loss: 0.3692, accu: 0.9261
Epoch 98/99


valid: 100%|██████████| 493/493 [00:09<00:00, 52.39it/s,  loss: 0.0015, accu: 1.0000]


Summary epoch:
 - Train loss: 0.0089, accu: 0.9972
 - Valid loss: 0.0924, accu: 0.9821
Epoch 99/99


valid: 100%|██████████| 493/493 [00:09<00:00, 52.43it/s,  loss: 0.0023, accu: 1.0000]

Summary epoch:
 - Train loss: 0.0082, accu: 0.9974
 - Valid loss: 0.0690, accu: 0.9860
